In [1]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.contrib.deprecation import deprecated
from dgl.nn.pytorch import Set2Set, NNConv
from utils import MPNNModel
import numpy as np
import time
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from dgl import model_zoo,DGLGraph
import math
import networkx as nx
import matplotlib.pyplot as plt
from utils_mpnn import Meter, set_random_seed, collate, EarlyStopping, load_model,load_brl_dataset,regress#,run_a_train_epoch,run_an_eval_epoch
import argparse
# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections
import sys; sys.argv=['']; del sys
parser = argparse.ArgumentParser(description='Molecule Regression')
parser.add_argument('-m', '--model', type=str,default='MPNN',help='Model to use')#choices=['MPNN', 'SCHNET', 'MGCN', 'AttentiveFP'],
#parser.add_argument('-d', '--dataset', type=str, default='bridge',help='Dataset to use')#choices=['Alchemy', 'Aromaticity'],                
parser.add_argument('-p', '--pre-trained', action='store_true', default=False, help='Whether to skip training and use a pre-trained model')
args = parser.parse_args().__dict__
training_setting= {
    'random_seed': 0,
    'batch_size': 64,#
    'num_epochs': 600,#900,
    'node_in_feats': 7,
    'edge_in_feats': 6,
    'output_dim': 120,
    'lr': 0.0001,#0.001,#
    'patience': 300,#20,
    'metric_name': 'l1',#'roc_auc',#
    'weight_decay': 0,
    'n_task':120,
}
args.update(training_setting)
args['device'] = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
args['data_path_x']='data/data_mpnn/'
args['data_path_label']='data/label/'
set_random_seed(args['random_seed'])

print(torch.cuda.current_device())
torch.cuda.set_device(1)
print(torch.cuda.current_device())

train_loader,val_loader,test_loader=load_brl_dataset(args)#for batch_id, batch_data in enumerate(train_loader):bg, labels = batch_data;print(a)
model = MPNNModel(node_input_dim=args['node_in_feats'],
                  edge_input_dim=args['edge_in_feats'],
                 output_dim=args['output_dim'])
if args['model'] in ['SCHNET', 'MGCN']:
    model.set_mean_std(train_set.mean, train_set.std, args['device'])
loss_fn1 =nn.BCEWithLogitsLoss(reduction='sum')#nn.KLDivLoss()#
#loss_fn1=nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
stopper = EarlyStopping(mode='lower', patience=args['patience'], filename='model_saved/bce/early_stop.pth')
model.to(args['device'])

def run_a_train_epoch_custom(args, epoch, model, data_loader,
                      loss_criterion1, optimizer):
    model.train()
    train_meter = Meter()
    correct=0
    total=0
    for batch_id, batch_data in enumerate(data_loader):
        bg, labels = batch_data
        labels= labels.to(args['device'])
        prediction = regress(args, model, bg)
        loss = (loss_criterion1(prediction, labels).float()).mean()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_meter.update(prediction, labels)
        
        _, predicted = torch.min(prediction.data, 1)
        total += labels.size(0)
        _, labels = torch.min(labels.data, 1)
        correct += (predicted == labels).sum().item();del predicted
        
    total_score_acc=100-100 * correct / total
    total_score = np.mean(train_meter.compute_metric(args['metric_name']))
    print('epoch {:d}/{:d}, training {} {:.4f} / accuracy(%) {:.4f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], total_score*1000, total_score_acc)) 
    #print('epoch {:d}/{:d}, training total_score_acc {:.4f}'.format(epoch + 1, args['num_epochs'], total_score_acc)) 
    
def run_an_eval_epoch_custom(args, model, data_loader):
    model.eval()
    eval_meter = Meter()
    correct=0
    total=0
    with torch.no_grad():
        for batch_id, batch_data in enumerate(data_loader):
            bg, labels = batch_data
            labels = labels.to(args['device'])
            prediction = regress(args, model, bg)
            eval_meter.update(prediction, labels)
            
            _, predicted = torch.min(prediction.data, 1)
            
            total += labels.size(0)
            _, labels = torch.min(labels.data, 1)
            correct += (predicted == labels).sum().item();del predicted
        total_score_acc=100-100 * correct / total
        total_score = np.mean(eval_meter.compute_metric(args['metric_name']))
    return total_score,total_score_acc

Using backend: pytorch
/home/shs/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


0


__main__:103: DeprecationWarning: The class is deprecated and will be removed from dgl in v0.5. Import MPNNPredictor from dgllife.model instead.


In [ ]:
for epoch in range(args['num_epochs']):
    st=time.time()
    # Train
    run_a_train_epoch_custom(args, epoch, model, train_loader, loss_fn1,optimizer)
    # Validation and early stop
    val_score,val_acc = run_an_eval_epoch_custom(args, model, val_loader)
    early_stop = stopper.step(val_acc, model,optimizer)
    print('epoch {:d}/{:d}, validation {} {:.4f}, accuracy(%) {:.4f}, best validation {} {:.4f}, time{:.1f}'.format(
        epoch + 1, args['num_epochs'], args['metric_name'], 
        val_score*1000,val_acc,args['metric_name'], stopper.best_score,time.time()-st))
    
torch.save({'model_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}, 'model_saved/bce/1.pth')

In [ ]:
state=torch.load('model_saved/bce/early_stop.pth') 
model.load_state_dict(state['model_state_dict'])
optimizer.load_state_dict(state['optimizer'])

for param_group in optimizer.param_groups: print(param_group['lr'])
for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
for param_group in optimizer.param_groups: print(param_group['lr'])

In [ ]:

model.eval()
#eval_meter = Meter()
y_p=[];y_t=[]
with torch.no_grad():
    for batch_id, batch_data in enumerate(test_loader):
        bg, labels = batch_data
        labels = labels.to(args['device'])  
        y_t.append(labels.cpu().detach().numpy())
        prediction = regress(args, model, bg)
        y_p.append(prediction.cpu().detach().numpy())

pred=np.concatenate(y_p,axis=0)
actual=np.concatenate(y_t,axis=0)

y_p=np.min(pred,axis=1)
y_t=np.min(actual,axis=1)
l_p=np.argmin(pred,axis=1)
l_t=np.argmin(actual,axis=1)
print(y_p.shape,y_t.shape)


import sklearn.metrics as metrics
#y_p=p_area;y_t=t_area;
mae = metrics.mean_absolute_error(y_t, y_p)
mse = metrics.mean_squared_error(y_t, y_p)
rmse=np.sqrt(mse)
print('mae', mae, '| rmse:',rmse)
print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
print(np.corrcoef(y_t.flatten(),y_p.flatten()))
print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
print('mape',np.mean(np.abs(y_t-y_p)/y_t))
iqr= np.subtract(*np.percentile(y_t, [75, 25]))
print('rmse/iqr',rmse/iqr)
print('rmse/mean',rmse/np.mean(y_t))
print('actual min max', np.min(y_t),np.max(y_t))
print('pred min max', np.min(y_p),np.max(y_p))

plt.plot(y_t.flatten(),y_p.flatten(),'.',color='darkblue',alpha=0.8)
plt.ylim(np.min([y_p,y_t]), np.max([y_p,y_t]))
plt.xlim(np.min([y_p,y_t]), np.max([y_p,y_t]))
plt.xlabel('actual')
plt.ylabel('pred')
plt.grid()
#plt.savefig('results/testset2.png')
plt.show()